In [1]:
from importlib import reload
import sys
import util
reload(util)
data = util.FontData
FontDataset = util.FontDataset
import experiment
reload(experiment)
from sklearn import linear_model, neural_network

In [2]:
data.load(knn_path="data/knn_dataset_weighted_kulah_cd.csv")

Loading embeddings...done
Loading typographic + semantic vectors...done
Loading images...done
Loading glyphs...done
Loading SVGs...done


In [3]:
train_dataset = FontDataset(data, 'train')
val_dataset = FontDataset(data, 'val')

In [4]:
train_typographic = data.get_all_typographic('train')
val_typographic = data.get_all_typographic('val')

In [5]:
train_image = data.get_all_image('train')
train_image = train_image.reshape(train_image.shape[0], -1)
val_image = data.get_all_image('val')
val_image = val_image.reshape(val_image.shape[0], -1)

In [6]:
train_semantic = data.get_all_semantic('train')
val_semantic = data.get_all_semantic('val')

### Linear Regression: Image -> Typographic

In [7]:
image_typographic_reg = linear_model.LinearRegression()

In [8]:
image_typographic_reg.fit(train_image, train_typographic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [9]:
train_stats, train_typographic_pred = experiment.run_experiment(image_typographic_reg, train_image, train_typographic)

{'RMSE': 1.6190169844776392e-06, 'R2': 0.9999999999555459}


In [27]:
import numpy as np
np.sum(train_image[0] == train_image[1])

41685

In [10]:
train_typographic.shape

(1303, 6)

In [11]:
val_stats, val_typographic_pred = experiment.run_experiment(image_typographic_reg, val_image, val_typographic)

{'RMSE': 0.14403529465198517, 'R2': 0.5955860215197114}


### Linear Regression: Typographic -> Semantic

In [12]:
typographic_semantic_reg = linear_model.LinearRegression()

In [13]:
typographic_semantic_reg.fit(train_typographic, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [14]:
train_stats, train_semantic_pred = experiment.run_experiment(typographic_semantic_reg, train_typographic, train_semantic)

{'RMSE': 0.10423894971609116, 'R2': 0.5348465153332079}


In [15]:
val_stats, val_semantic_pred = experiment.run_experiment(typographic_semantic_reg, val_typographic, val_semantic)

{'RMSE': 0.10544156283140182, 'R2': 0.551798933862579}


### Linear Regressions: Image -> Typographic -> Semantic

In [16]:
class SequentialLinearRegression:
    def __init__(self, reg1, reg2):
        self.reg1 = reg1
        self.reg2 = reg2
    def predict(self, X):
        return self.reg2.predict(self.reg1.predict(X))

image_typographic_semantic_reg = SequentialLinearRegression(image_typographic_reg, typographic_semantic_reg)

In [17]:
train_semantic

array([[0.3937207 , 0.26580928, 0.44323922, ..., 0.51104709, 0.61849329,
        0.53933688],
       [0.44119959, 0.36253881, 0.30584526, ..., 0.69577073, 0.50729707,
        0.52776776],
       [0.24112255, 0.08107704, 0.23196869, ..., 0.63499231, 0.48026325,
        0.50188639],
       ...,
       [0.76294362, 0.69486117, 0.64503608, ..., 0.41120683, 0.86035076,
        0.45234435],
       [0.74068568, 0.59511034, 0.41721934, ..., 0.63478506, 0.46328393,
        0.32634286],
       [0.72045128, 0.83274388, 0.5092478 , ..., 0.83010049, 0.56611526,
        0.24808567]])

In [18]:
train_stats, train_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, train_image, train_semantic)

{'RMSE': 0.10423890501260757, 'R2': 0.5348468170071726}


In [19]:
val_stats, val_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, val_image, val_semantic)

{'RMSE': 0.12145368754863739, 'R2': 0.4203832187871161}


### Linear Regression: Image -> Semantic

In [20]:
image_semantic_reg = linear_model.LinearRegression()

In [21]:
image_semantic_reg.fit(train_image, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [22]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_reg, train_image, train_semantic)

{'RMSE': 1.4079495258556562e-06, 'R2': 0.9999999999119905}


In [29]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_reg, val_image, val_semantic)

{'RMSE': 0.10856691002845764, 'R2': 0.553380888114271}


In [32]:
np.matmul(image_semantic_reg.coef_, train_image[0]) + image_semantic_reg.intercept_

array([0.39371884, 0.2658074 , 0.44323844, 0.5499176 , 0.11449365,
       0.36957505, 0.8596678 , 0.68012685, 0.0854581 , 0.26743853,
       0.4057957 , 0.12494516, 0.30007732, 0.86702925, 0.94195163,
       0.7639547 , 0.9478284 , 0.67611986, 0.4997297 , 0.9002953 ,
       0.4423493 , 0.2628653 , 0.33460957, 0.8297703 , 0.18913275,
       0.75447553, 0.5629937 , 0.485982  , 0.51104844, 0.6184929 ,
       0.5393361 ], dtype=float32)

In [33]:
train_semantic[0]

array([0.3937207 , 0.26580928, 0.44323922, 0.5499175 , 0.11449732,
       0.36957753, 0.85966531, 0.68012569, 0.0854604 , 0.26744044,
       0.40579383, 0.12494763, 0.30007925, 0.86702582, 0.94195076,
       0.76395219, 0.9478269 , 0.67611948, 0.4997272 , 0.90029394,
       0.44235098, 0.26286693, 0.33461168, 0.82976761, 0.18913256,
       0.75447244, 0.56299514, 0.48598331, 0.51104709, 0.61849329,
       0.53933688])

In [35]:
image_semantic_reg.coef_

array([[-1.04370818e-04,  6.13906886e-05, -1.78799964e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.00814102e-05, -2.82585388e-05, -1.02749094e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.18792586e-04,  1.49730826e-04, -1.98266469e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-2.42962851e-05,  2.39128713e-05,  3.73900402e-05, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.14406314e-04,  7.69624021e-05, -1.16322190e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.82972290e-05, -1.12727867e-05,  2.77321087e-05, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]], dtype=float32)

### Multilayer Perceptron: Image -> Semantic

In [ ]:
image_semantic_nn = neural_network.MLPRegressor(hidden_layer_sizes=(10,), max_iter=500)

In [ ]:
image_semantic_nn.fit(train_image, train_semantic)

In [ ]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_nn, train_image, train_semantic)

In [33]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_nn, val_image, val_semantic)

{'RMSE': 0.1747315675020218, 'R2': -0.3753219676038256}


#### Example